*Import necessary Modules*

In [166]:
import os
import random
import shutil
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Add, Activation
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt

*Mount drive for a dataset*

In [167]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


*Extract RAR to folder*

In [168]:
rar_path = '/content/drive/My Drive/CUDA-dataset/TrashNet.rar'
extract_path = '/content/dataset'

os.makedirs(extract_path, exist_ok=True)

!unrar x -o+ "{rar_path}" "{extract_path}"


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/My Drive/CUDA-dataset/TrashNet.rar

Creating    /content/dataset/TrashNet/cardboard                       OK
Extracting  /content/dataset/TrashNet/cardboard/cardboard1.jpg             0%  OK 
Extracting  /content/dataset/TrashNet/cardboard/cardboard10.jpg            0%  OK 
Extracting  /content/dataset/TrashNet/cardboard/cardboard100.jpg           0%  OK 
Extracting  /content/dataset/TrashNet/cardboard/cardboard101.jpg           0%  OK 
Extracting  /content/dataset/TrashNet/cardboard/cardboard102.jpg           0%  OK 
Extracting  /content/dataset/TrashNet/cardboard/cardboard103.jpg           0%  OK 
Extracting  /content/dataset/TrashNet/cardboard/cardboard104.jpg           0%  OK 
Extracting  /content/dataset/TrashNet/cardboard/cardboard105.jpg           0%  OK 
Extracting  /content/dataset/TrashNet/cardboard/

*Split data to Train and Validation sets*

In [169]:
shutil.rmtree('dataset/TrashNet/train')
shutil.rmtree('dataset/TrashNet/validation')
base_dir = 'dataset/TrashNet'
classes = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

# Podział danych na zbiór treningowy i walidacyjny
split_ratio = 0.8  # 80% trening, 20% walidacja

for cls in classes:
    class_dir = os.path.join(base_dir, cls)
    images = os.listdir(class_dir)
    random.shuffle(images)

    split_point = int(split_ratio * len(images))
    train_images = images[:split_point]
    validation_images = images[split_point:]

    for img in train_images:
        source = os.path.join(class_dir, img)
        destination = os.path.join(train_dir, cls, img)
        os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
        shutil.move(source, destination)

    for img in validation_images:
        source = os.path.join(class_dir, img)
        destination = os.path.join(validation_dir, cls, img)
        os.makedirs(os.path.join(validation_dir, cls), exist_ok=True)
        if img not in train_images:  # Sprawdź, czy obraz nie został już użyty w zbiorze treningowym
            shutil.move(source, destination)

*Removing existing directories - after splitting*

In [170]:
base_path = '/content/dataset/TrashNet'
for cls in classes:
    folder_path = os.path.join(base_path, cls)
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)

*Prepare Data Generators*

In [171]:
# Ustawienia
img_height = 150
img_width = 150
batch_size = 32
epochs = 5

In [172]:
# Przygotowanie danych
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [173]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 2019 images belonging to 6 classes.
Found 508 images belonging to 6 classes.


*Creation of a simple CNN Model*

In [174]:
# Tworzenie modelu
num_classes = len(train_generator.class_indices)

model_simpleCNN = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])


In [175]:
model_simpleCNN.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_simpleCNN.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_58 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_48 (MaxPooli  (None, 74, 74, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_59 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_49 (MaxPooli  (None, 36, 36, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_60 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_50 (MaxPooli  (None, 17, 17, 128)     

In [176]:
history_simpleCNN = model_simpleCNN.fit(
    # provides batches of samples to train on each epoch
    train_generator,

    # train_generator.samples - total number of training samples
    # it calculates the number of steps needed to process the entire training data sets once in each epoch
    steps_per_epoch=train_generator.samples // batch_size,

    epochs=10,

    #  the Model will evaluate the performance on this validation data
    validation_data=validation_generator,

    validation_steps=validation_generator.samples // batch_size
)


Epoch 1/10
63/63 [==============================] - 139s 2s/step - loss: 1.6983 - accuracy: 0.2939 - val_loss: 1.4519 - val_accuracy: 0.4021
Epoch 2/10
63/63 [==============================] - 136s 2s/step - loss: 1.4301 - accuracy: 0.4127 - val_loss: 1.3452 - val_accuracy: 0.4333
Epoch 3/10
63/63 [==============================] - 132s 2s/step - loss: 1.2905 - accuracy: 0.4836 - val_loss: 1.3776 - val_accuracy: 0.4354
Epoch 4/10
63/63 [==============================] - 136s 2s/step - loss: 1.2550 - accuracy: 0.4917 - val_loss: 1.2627 - val_accuracy: 0.4604
Epoch 5/10
63/63 [==============================] - 135s 2s/step - loss: 1.1855 - accuracy: 0.5189 - val_loss: 1.2724 - val_accuracy: 0.4896
Epoch 6/10
63/63 [==============================] - 133s 2s/step - loss: 1.1029 - accuracy: 0.5687 - val_loss: 1.1926 - val_accuracy: 0.5396
Epoch 7/10
63/63 [==============================] - 134s 2s/step - loss: 1.1161 - accuracy: 0.5682 - val_loss: 1.1417 - val_accuracy: 0.5771
Epoch 8/10
63

*save Models*

In [177]:
model_simpleCNN.save('simpleCNN.h5')
# model_ResNet.save('ResNet.h5')